Universidad Galileo

Text Mining and Image Recognition

PAPD - Sección V

Sergio José Barrios Martínez

Carnet No. 19012765

## Laboratorio No. 2


***Instrucciones:*** 
A continuación verá una lista de ejercicios que debe completar para poder entregar el laboratorio #2. Al analizar, todos sus archivos deben estar contenidos en un archivo lab2-sucarnet.zip. Este archivo debe entregarlo en el link del GES. Por favor cree una carpeta para cada
ejercicio que usted realice.

In [369]:
import pandas as pd
import spacy
import spacy.cli
import re
import nltk
import numpy as np
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
import pickle

### **Problema #2:**
Para el datasate llamado **imdb_rev.txt** desarrolle un clasificador de reviews positivos o negativos, la variable que debe predecir está codificada como 0 y como 1, donde 0 es un review malo y 1 es un review bueno. Tome en cuenta que este archivo no es un csv y usted deberá procesarlo para poder formar una estructura adecuada para la clasificación, ademas note que está en ingles, por tal motivo debe utilizar el modelo de lenguaje adecuado. Recuerde usar todos los recursos que conoce sobre Machine Learning para determinar si su programa clasifica adecuadamente, puede usar cualquier clasificador que desee incluso varios si lo considera necesario.

In [111]:
filename = 'imdb_rev.txt'
df = pd.read_csv(filename, sep='\t', lineterminator='\n',names=["Review","Rating"],engine='c')
df

,Review,Rating
0,"A very, very, very slow-moving, aimless movie ...",0
1,Not sure who was more lost - the flat characte...,0
2,Attempting artiness with black & white and cle...,0
3,Very little music or anything to speak of.,0
4,The best scene in the movie was when Gerardo i...,1
...,...,...
995,I just got bored watching Jessice Lange take h...,0
996,"Unfortunately, any virtue in this film's produ...",0
997,"In a word, it is embarrassing.",0
998,Exceptionally bad!,0


### Modelo en Idioma Inglés

In [12]:
spacy.cli.download('en_core_web_sm')
nlp = spacy.load('en_core_web_sm')

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


### Balance de Datos

In [112]:
positivo = df[df['Rating'] == 1]
negativo = df[df['Rating'] == 0]
positivo.shape, negativo.shape

((500, 2), (500, 2))

### Normalización

In [117]:
corpus = df.Review

In [114]:
def Normalizacion(corpus):
    newCorpus = []
    for doc in corpus:
        newCorpus.append(re.sub(r'[^a-zA-Z0-9\s{1}áéíóúü]','',doc).lower().strip().rstrip('\n').rstrip('\r\n').lstrip('"'))
  
    return newCorpus

In [115]:
corpusNormalizado = Normalizacion(corpus)

In [118]:
def tokenizacion(corpus):
    
    documents = []
    for doc in corpus:
        documents.append(nlp(doc))
  
    return documents

In [119]:
corpusTokenizado = tokenizacion(corpusNormalizado)

### Remoción de Stop-Words

In [121]:
def removeStops(corpus):
    document = []
    for doc in corpus:
        s = ""
        for token in doc:
            if(token.is_stop == False):
                s = s + token.text + " "
        document.append(s.strip())
    
    return document

In [122]:
corpusStopwords = removeStops(corpusTokenizado)

### Lematización

In [124]:
def stemmingLemmating(corpus):
    documents = tokenizacion(corpus)
    newDocs = []
    for doc in documents:
        s = ""
        for token in doc:
            s = s + token.lemma_ + " "
        newDocs.append(s.strip())
    return newDocs

In [125]:
corpusLematizado = stemmingLemmating(corpusStopwords)

### Cadena de elementos únicos a partir de Lista

In [127]:
def listToString(s):
    strX = ""
    for palabra in s:
        strX = strX + palabra + " "
    return strX

In [128]:
strCorpus = listToString(corpusLematizado)
strCorpus = strCorpus.strip().split(' ')
setCorpus = set(strCorpus)
setCorpus.remove('')

### Ensamble de DataFrame

In [129]:
def generateEmptyTf(cols,rows):
    outDf = pd.DataFrame(index=rows, columns=cols)
    outDf = outDf.fillna(0)

    return outDf

In [130]:
corpusCols = list(setCorpus)
corpusRows = range(0,len(corpusLematizado))
tfCorpus = generateEmptyTf(corpusCols,corpusRows)
tfCorpus

,fodder,punishment,savor,rivalry,thiswhen,deep,massive,moment,author,bop,...,store,rpg,appropriate,start,know,unrestrained,koteasjack,energetic,weird,wartime
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
996,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
997,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
998,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Cálculo de TF

In [131]:
def calculoTF(corpus,df):
    
    corpus = tokenizacion(corpus)
    for index, doc in enumerate(corpus):

        bagOfWordsLen = len(doc)
        
        for word in doc:
            try:
                colIndex = list(df.columns).index(word.text)
                df.iloc[index,colIndex] = df.iloc[index,colIndex] + 1
            except:
                pass
      
        if bagOfWordsLen != 0:
            df.iloc[index,:] = df.iloc[index,:]/bagOfWordsLen
        else:
            df.iloc[index,:] = 0
 
    return df

In [132]:
tfMatrix = calculoTF(corpusLematizado,tfCorpus)

In [133]:
tfMatrix

,fodder,punishment,savor,rivalry,thiswhen,deep,massive,moment,author,bop,...,store,rpg,appropriate,start,know,unrestrained,koteasjack,energetic,weird,wartime
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
997,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Cálculo de IDF

In [134]:
def calculoIDF(df):
    N = df.shape[0]
    valX = (N/df.astype(bool).sum(axis=0))
    idfValues = pd.Series(np.log(valX))
    
    return idfValues

In [135]:
corpusIDF = calculoIDF(tfMatrix)
corpusIDF

fodder          6.907755
punishment      6.907755
savor           6.907755
rivalry         6.907755
thiswhen        6.907755
                  ...   
unrestrained    6.907755
koteasjack      6.907755
energetic       6.907755
weird           6.214608
wartime         6.907755
Length: 2603, dtype: float64

In [136]:
tfIdfCorpus = tfMatrix.mul(corpusIDF,axis=1)
tfIdfCorpus = tfIdfCorpus.fillna(0)
tfIdfCorpus

,fodder,punishment,savor,rivalry,thiswhen,deep,massive,moment,author,bop,...,store,rpg,appropriate,start,know,unrestrained,koteasjack,energetic,weird,wartime
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
997,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Entrenamiento Modelo

In [169]:
X_train, X_test, y_train, y_test = train_test_split(tfIdfCorpus, df['Rating'],
                                                    test_size = 0.3, random_state=0, shuffle=True)

### Modelo Random Forest

In [365]:
rfc = RandomForestClassifier(max_depth=15, min_samples_leaf=2, random_state=0)
rfc.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=15, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=2, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [366]:
y_preds_rfc = rfc.predict(X_test)

In [367]:
print(np.around(accuracy_score(y_test, y_preds_rfc)*100.0,2),"%")

66.33 %


### Modelo SVM

In [286]:
svmc = SVC(kernel='linear',C=0.075)
svmc.fit(X_train,y_train)

SVC(C=0.075, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [287]:
y_preds_svm = svmc.predict(X_test)

In [288]:
print(np.around(accuracy_score(y_test, y_preds_svm)*100.0,2),"%")

79.67 %


### Modelo Regresión Logística

In [341]:
lrc = LogisticRegression(random_state=0,C=0.2)
lrc.fit(X_train,y_train)

LogisticRegression(C=0.2, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=0, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [342]:
y_preds_lr = lrc.predict(X_test)

In [343]:
print(np.around(accuracy_score(y_test, y_preds_lr)*100.0,2),"%")

80.0 %


### Modelo K-Nearest Neighbors

In [442]:
knn = KNeighborsClassifier(n_neighbors=20)
knn.fit(X_train,y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=20, p=2,
                     weights='uniform')

In [443]:
y_preds_knn = knn.predict(X_test)

In [444]:
print(np.around(accuracy_score(y_test, y_preds_knn)*100.0,2),"%")

68.0 %
